# Consensus

**Contributed by**: Benoît Legat

In this notebook, we show how to apply the JSR theory to compute the rate of convergence of agents to consensus.
This example is [Example 2.52, P17].

[P17] M. Philippe.
*Path-Complete Methods and Analysis of Constrained Switching Systems*
Doctoral dissertation, UCLouvain, **2017**

In [1]:
using LinearAlgebra
P = [ 1.0  0.0  1.0  2.0
     -1.0  0.0  1.0  2.0
      0.0  1.0  0.0 -3.0
      0.0 -1.0  0.0 -3.0
      0.0  0.0 -2.0  2.0]
for i in 1:4
    P[:, i] /= norm(P[:, i])
end
@show P' * ones(5)
round.(P' * P, digits=16)

P' * ones(5) = [0.0, 0.0, 0.0, 0.0]


4×4 Matrix{Float64}:
 1.0   0.0   0.0   0.0
 0.0   1.0   0.0  -0.0
 0.0   0.0   1.0  -0.0
 0.0  -0.0  -0.0   1.0

We build the switched system as follows:

In [2]:
using SwitchOnSafety
H1 = [0.2 0.8 0   0   0
      0   0.2 0.8 0   0
      0   0   0.2 0.8 0
      0   0   0   0.2 0.8
      0.2 0   0   0   0.8]
A1 = P' * H1 * P
H2 = [1   0   0   0   0
      0.8 0.2 0   0   0
      0   0   1   0   0
      0   0   0.8 0.2 0
      0   0   0   0   1]
A2 = P' * H2 * P
H3 = [1   0   0   0   0
      0   1   0   0   0
      0   0.8 0.2 0   0
      0   0   0   1   0
      0   0   0   0.2 0.8]
A3 = P' * H3 * P
function automaton(N)
    a = GraphAutomaton(N) # See [P17, Figure 2.17] for what automaton(3) should be
    add_transition!(a, 1, 1, 1) # Node i means, H1 was used i-1 steps ago
    for i in 2:N
        add_transition!(a, i-1, i, 2)
        add_transition!(a, i-1, i, 3)
        add_transition!(a, i, 1, 1)
    end
    return a
end
ss(N) = discreteswitchedsystem([A1, A2, A3], automaton(N))

ss (generic function with 1 method)

Pick an SDP solver from [this list](https://jump.dev/JuMP.jl/stable/installation/#Supported-solvers).

In [3]:
import CSDP
optimizer_constructor = optimizer_with_attributes(CSDP.Optimizer, MOI.Silent() => true);

# $N = 1$

With $N = 1$, the system is not switched, there is only one matrix and the JSR is approximately $0.7273$.

In [4]:
ρ(A1)

0.7273364816948454

With common quadratic Lyapunov functions, we get:

In [5]:
soslyapb(ss(1), 1; optimizer_constructor=optimizer_constructor, tol=1e-7, verbose=1)

┌ Warning: Solver returned with status : (MathOptInterface.ALMOST_OPTIMAL, MathOptInterface.NEARLY_FEASIBLE_POINT, MathOptInterface.NEARLY_FEASIBLE_POINT, "Problem solved to near optimality.") for γ=0.7273334531829795, (MathOptInterface.ALMOST_OPTIMAL, MathOptInterface.NEARLY_FEASIBLE_POINT, MathOptInterface.NEARLY_FEASIBLE_POINT, "Problem solved to near optimality.") for γ=0.7273344936425743 and (MathOptInterface.ALMOST_OPTIMAL, MathOptInterface.NEARLY_FEASIBLE_POINT, MathOptInterface.NEARLY_FEASIBLE_POINT, "Problem solved to near optimality.") for γ=0.7273355341036575. Stopping bisection with 5.722045898381989e-6 > 1.0e-7 (= tol)
└ @ SwitchOnSafety ~/work/SwitchOnSafety.jl/SwitchOnSafety.jl/src/sos.jl:234


(0.727332412724873, 0.7273365745662291)

# $N = 2$

In [6]:
ss2 = ss(2);

We start with CQLF (Common Quadratic Lyapunov Function)

In [7]:
soslyapb(ss2, 1; optimizer_constructor=optimizer_constructor, tol=1e-7, verbose=1)

Lower bound: 0.9871584096980368
Upper bound: 0.9871584685372649
Log-diff   : 5.9604644719879474e-8 ≤ 1.0e-7


(0.7760559438489385, 0.9871584685372649)

We use rounding to obtain good lower bounds:

In [8]:
@time seq = sosbuildsequence(ss2, 1, niter=100, l=1, p_0=:Primal)
@time psw = findsmp(seq)

  1.417008 seconds (2.92 M allocations: 162.267 MiB, 3.31% gc time, 99.06% compilation time)
  0.327701 seconds (498.73 k allocations: 34.467 MiB, 95.66% compilation time)


PSW(0.9262308830354682, [3, 1, 3, 1, 3, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1])

We now try with quartic polynomials

In [9]:
soslyapb(ss2, 2; optimizer_constructor=optimizer_constructor, tol=1e-5, verbose=1)

Lower bound: 0.9270886518320397
Upper bound: 0.9270969438351278
Log-diff   : 8.944091796941889e-6 ≤ 1.0e-5


(0.8220048027269916, 0.9270969438351278)

Use rounding:

In [10]:
@time seq = sosbuildsequence(ss2, 2, niter=100, l=2, p_0=:Primal)
@time psw = findsmp(seq)

  0.018653 seconds (205.11 k allocations: 15.407 MiB)
  0.015092 seconds (43.68 k allocations: 10.763 MiB)


PSW(0.9262308830354682, [3, 1, 3, 1, 3, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1])

Sextic:

In [11]:
soslyapb(ss2, 3; optimizer_constructor=optimizer_constructor, tol=1e-3, verbose=1)

Lower bound: 0.9262308830354682
Upper bound: 0.9271358471872897
Log-diff   : 0.0009765625000000139 ≤ 0.001


(0.8548461972141421, 0.9271358471872897)

Rounding:

In [12]:
@time seq = sosbuildsequence(ss2, 3, niter=45, l=1, p_0=:Primal)
@time psw = findsmp(seq)

  0.057647 seconds (352.65 k allocations: 32.358 MiB)
  0.002385 seconds (8.30 k allocations: 1.591 MiB)


PSW(0.9249138169675055, [3, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 3, 1, 3, 1])

# $N = 3$

In [13]:
soslyapb(ss(3), 1; optimizer_constructor=optimizer_constructor, tol=1e-7, verbose=1)

Lower bound: 1.0738022226700241
Upper bound: 1.073802286673626
Log-diff   : 5.96046448031462e-8 ≤ 1.0e-7


(0.7917709743371962, 1.073802286673626)

# $N = 4$

In [14]:
soslyapb(ss(4), 1; optimizer_constructor=optimizer_constructor, tol=1e-7, verbose=1)

Lower bound: 1.096910401582472
Upper bound: 1.0969104996539085
Log-diff   : 8.940696706594142e-8 ≤ 1.0e-7


(0.7900725616728572, 1.0969104996539085)

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*